<a href="https://colab.research.google.com/github/pe44enka/TaxiFarePrediction/blob/master/TaxiFarePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYC Taxi Fare Prediction**

![](https://static.vecteezy.com/system/resources/previews/000/118/272/original/free-new-york-taxi-watercolor-vector.jpg)


### **Objectives**

Imagine you are in Big Apple. New to town and have no clue how to get from Central park to Empire State Building. After some useless attemps and short fair buttle you finally got your cab. But hey! How much is it gonna cost you in this crazy city?


---

### **Goal of the project**
To predict the fare amount (inclusive of tolls) for a taxi ride in New York City given the pickup and dropoff locations.

### **Data**
[New York City Taxi Fare Prediction](https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction) dataset available at Kaggle as part of competition.

The dataset contains the following fields:

Field name | Description
--- |--- 
*key* | identifier for each trip
*fare_amount* | the cost of each trip in usd
*pickup_datetime* | date and time when the meter was engaged
*passenger_count* | the number of passengers in the vehicle (driver entered value)
*pickup_longitude* | the longitude where the meter was engaged
*pickup_latitude* | the latitude where the meter was engaged
*dropoff_longitude* | the longitude where the meter was disengaged
*dropoff_latitude* | the latitude where the meter was disengaged

### **Techniques**
In this project we will use:
* **Data preprocessing**: SelectFromModel, SimpleImputer, OneHotEncoder, StandardScaler, ColumnTransformer, pandas.get_dummies
* **ML algorihms**: LinearRegression, DecisionTreeRegressor RandomForestRegressor, GradientBoostingRegressor, XGBRegressor 
* **Hyperparameter turning:** GridSearchCV
* **Model training/applying:** Pipeline, tran_test_split


---

# Load Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from geopy.distance import great_circle

# Load Data

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/pe44enka/TaxiFarePrediction/master/data/train.csv')
print('df.shape: ', df.shape)
df.head()

df.shape:  (1048575, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,26:21.0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


# Data Cleaning

## Overview


Before playing with ML models and trying to predict anything let's get ourselves familiar with data we have.

---



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   key                1048575 non-null  object 
 1   fare_amount        1048575 non-null  float64
 2   pickup_datetime    1048575 non-null  object 
 3   pickup_longitude   1048575 non-null  float64
 4   pickup_latitude    1048575 non-null  float64
 5   dropoff_longitude  1048565 non-null  float64
 6   dropoff_latitude   1048565 non-null  float64
 7   passenger_count    1048575 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 64.0+ MB



---

**Notes:** there are mixed categorical and numerical features as well as missing values in the data

**Conclusion:** need to get rid of missing values and to deal with categorical data

---



## Missing Values

Let's have a look on features with missing values.

---

In [4]:
df[df.columns[df.isnull().sum().values>0]].isna().sum()

dropoff_longitude    10
dropoff_latitude     10
dtype: int64



---

As we can see ```dropoff_longitude``` and ```dropoff_latitude``` have missing values.
As long as it's unavailable to fill these gaps with any known values and amount of missing values is not high in comparison with all entities in the data (<0.001%), we can skip these entities with dropping the whole row. 


---



In [5]:
df.dropna(axis=0, inplace=True) #drop rows with NaN
df.reset_index(drop=True, inplace=True) #reset index after dropping rows
df.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [6]:
print('df.shape: ', df.shape)
df.head()

df.shape:  (1048565, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,26:21.0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [7]:
df[df.columns[df.isnull().sum().values>0]].isna().sum()

Series([], dtype: float64)



---

There is no missing values in the data.

---



## Coordinate Columns Values

Before getting distance of rides out of coordinate columns (```'pickup_latitude```, ```pickup_longitude```, ```dropoff_latitude``` and ```dropoff_longitude```) we need to check if their values lie in range (-90, 90) degrees. Let's check descriptive statistic on these columns.

---

In [8]:
df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']].describe()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,1.048565e+06,1.048565e+06,1.048565e+06,1.048565e+06
mean,3.993097e+01,-7.252729e+01,3.992496e+01,-7.252750e+01
std,7.725744e+00,1.200783e+01,8.529585e+00,1.141154e+01
min,-3.116285e+03,-3.377681e+03,-3.114339e+03,-3.383297e+03
25%,4.073496e+01,-7.399207e+01,4.073406e+01,-7.399138e+01
50%,4.075267e+01,-7.398180e+01,4.075318e+01,-7.398014e+01
75%,4.076714e+01,-7.396711e+01,4.076812e+01,-7.396367e+01
max,2.621628e+03,2.522271e+03,1.989728e+03,1.717003e+03



---

As we can see all coordinate columns have *min* and *max* values far away from -90 and 90 where they are supposed to be. Let's check how many entities have coordinates lower than -90 and higher than 90.

---



In [9]:
#concatenate parts of the data cutted accordingly to the conditions described above
coor_df = pd.concat([df[df.pickup_latitude < - 90], df[df.pickup_latitude > 90],
                     df[df.pickup_longitude < - 90], df[df.pickup_longitude > 90],
                     df[df.dropoff_latitude < - 90], df[df.dropoff_latitude > 90],
                     df[df.dropoff_longitude < - 90], df[df.dropoff_longitude > 90]
                     ])
coor_df = coor_df.drop_duplicates() #remove duplicated rows
coor_df.shape

(49, 8)



---

There are a few values in coordinate columns that do not follow the condition. These amount is not high in comparison with all entities in the data (<0.005%), we can skip these entities with dropping the whole row.

---



In [10]:
df.drop(index=coor_df.index.to_list(), inplace=True)
print('df.shape: ', df.shape)
df.head()

df.shape:  (1048516, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,26:21.0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [11]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.048516e+06,1.048516e+06,1.048516e+06,1.048516e+06,1.048516e+06,1.048516e+06
mean,1.134531e+01,-7.251788e+01,3.992727e+01,-7.251708e+01,3.992701e+01,1.684913e+00
std,9.819766e+00,1.039017e+01,6.086952e+00,1.039004e+01,6.089382e+00,1.323140e+00
min,-4.490000e+01,-8.943979e+01,-7.401659e+01,-8.680412e+01,-7.403520e+01,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073496e+01,-7.399138e+01,4.073406e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075267e+01,-7.398014e+01,4.075318e+01,1.000000e+00
75%,1.250000e+01,-7.396711e+01,4.076714e+01,-7.396367e+01,4.076812e+01,2.000000e+00
max,5.000000e+02,4.085036e+01,6.940000e+01,4.558162e+01,8.151018e+01,2.080000e+02




---
All coordinates are in required range.

Let's also check the data with null coordinates.

---



In [12]:
null_coor_df = pd.concat([df[df.pickup_latitude == 0], df[df.pickup_longitude == 0],
                          df[df.dropoff_latitude == 0], df[df.dropoff_longitude == 0]])
                    
null_coor_df = null_coor_df.drop_duplicates() #remove duplicated rows
null_coor_df.shape

(20595, 8)



---

There are a lot of entities with zero coordinates what means it brings us no information about the distance of particular rides and makes this entities senseless and potentially dangeroud for future model training as far as it can lead to wrong predictions. 

We will treat these zero values as missing values. There is no way we can find info and fill them with certain values. And it's about 2% of the data we will drop them anyway to avoid wrong model training. 

---



In [13]:
df.drop(index=null_coor_df.index.to_list(), inplace=True)
print('df.shape: ', df.shape)
df.head()

df.shape:  (1027921, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,26:21.0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [14]:
null_coor_df = pd.concat([df[df.pickup_latitude == 0], df[df.pickup_longitude == 0],
                          df[df.dropoff_latitude == 0], df[df.dropoff_longitude == 0]])
                    
null_coor_df = null_coor_df.drop_duplicates() #remove duplicated rows
null_coor_df.shape

(0, 8)



---

No entities with zero coordinates left.

---



## Feature Engineering

Let's transform data features to make them more useful for future modeling.

---

### Columns Dropping

We will drop column ```key``` as far as it's just indicator of the ride and brings no useful info for further analysis and modeling.

----

In [15]:
df.drop(columns=['key'], inplace=True) #remove key column
print('df.shape: ', df.shape)
df.head()

df.shape:  (1027921, 7)


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


### Column Creating

#### Datetime Columns



On next step we will parse ```pickup_datetime``` into several columns:
* year
* season
* month_name
* day
* day_name
* hour
* rush hour (yes/no): 7am - 10 am, 3pm - 7pm

By default all datetime based columns are considered as strings in pandas. So we need to convert string data to datetime features. And then extract from datetime object all information we need.

---



In [16]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
df.dtypes

fare_amount                      float64
pickup_datetime      datetime64[ns, UTC]
pickup_longitude                 float64
pickup_latitude                  float64
dropoff_longitude                float64
dropoff_latitude                 float64
passenger_count                    int64
dtype: object

In [17]:
df['year'] = df.pickup_datetime.dt.year #year

#getting seasons for each entity
seasons = ['Winter', 'Spring', 'Summer', 'Fall'] #season
df['season'] = [seasons[i-1] for i in (df.pickup_datetime.dt.month%12// 3 + 1).values]

df['month'] = df.pickup_datetime.dt.month_name() #month
df['day'] = df.pickup_datetime.dt.day #day
df['day_name'] = df.pickup_datetime.dt.day_name() #day name
df['hour'] = df.pickup_datetime.dt.hour #hour

#finding out if the ride was in rush hour (7am-10am, 3pm-7pm) or not
rush_hour = []
for i in df.hour.values:
  if i in range(7,11):
    rush_hour.append(1)
  elif i in range(15,20):
    rush_hour.append(1)
  else:
    rush_hour.append(0)
df['rush_hour'] = rush_hour

df.drop(columns=['pickup_datetime'], inplace=True) # drop donor column

print('df.shape: ', df.shape)
df.head()

df.shape:  (1027921, 13)


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,season,month,day,day_name,hour,rush_hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,Summer,June,15,Monday,17,1
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,Winter,January,5,Tuesday,16,1
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,Summer,August,18,Thursday,0,0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,Spring,April,21,Saturday,4,0
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,Spring,March,9,Tuesday,7,1


#### Distance Column



One of the most important metrics influncing on the taxi ride fare is distance. To get it we will need to convert latitude and longitude of pick up and drop off into km.

For this purpose we will use **Haversine (or great circle) distance** - the angular distance between two points on the surface of a sphere. The first coordinate of each point is assumed to be the latitude, the second is the longitude.

If speaking on Python we can do it by defining a custom function or by using different libraries:
* ```from geopy.distance import great_circle```
* ```sklearn.neighbors```
* ```from sklearn.metrics.pairwise import haversine_distances```


In this project i will use ```geopy``` library. Let's check how it works on the first ride in our data.

---



In [18]:
coordA = [df.pickup_latitude.iloc[0], df.pickup_longitude.iloc[0]]
coordB = [df.dropoff_latitude.iloc[0], df.dropoff_longitude.iloc[0]]
print ('Distance is: {:.3f} km'.format(float(great_circle(coordA, coordB).kilometers)))

Distance is: 1.031 km




---

Now when we know how to count the distance for 1 entity we just need to put the code in loop and get the distance for each of them. Let's do it!

---



In [19]:
distance = []
for i in range(df.shape[0]):
  coordA = [df.pickup_latitude.iloc[i], df.pickup_longitude.iloc[i]]
  coordB = [df.dropoff_latitude.iloc[i], df.dropoff_longitude.iloc[i]]
  distance.append(round(float(great_circle(coordA, coordB).kilometers), 3))

df['distance'] = distance #create a column with distance for each ride
#df.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], inplace=True) #drop donor columns

print('df.shape: ', df.shape)
df.head()

df.shape:  (1027921, 10)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999


## Exploratore Data Analysis

### EDA Overview

Once we've done with feature creating it's time to explore our data.

---

In [20]:
df.describe(include='all')

,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
count,1.027921e+06,1.027921e+06,1.027921e+06,1027921,1027921,1.027921e+06,1027921,1.027921e+06,1.027921e+06,1.027921e+06
unique,NaN,NaN,NaN,4,12,NaN,7,NaN,NaN,NaN
top,NaN,NaN,NaN,Spring,May,NaN,Friday,NaN,NaN,NaN
freq,NaN,NaN,NaN,287226,96323,NaN,158583,NaN,NaN,NaN
mean,1.133004e+01,1.685244e+00,2.011739e+03,NaN,NaN,1.569820e+01,NaN,1.350912e+01,4.353185e-01,4.395170e+00
std,9.746336e+00,1.307981e+00,1.862741e+00,NaN,NaN,8.680878e+00,NaN,6.511469e+00,4.957989e-01,7.550311e+01
min,-4.490000e+01,0.000000e+00,2.009000e+03,NaN,NaN,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,1.000000e+00,2.010000e+03,NaN,NaN,8.000000e+00,NaN,9.000000e+00,0.000000e+00,1.254000e+00
50%,8.500000e+00,1.000000e+00,2.012000e+03,NaN,NaN,1.600000e+01,NaN,1.400000e+01,0.000000e+00,2.153000e+00
75%,1.250000e+01,2.000000e+00,2.013000e+03,NaN,NaN,2.300000e+01,NaN,1.900000e+01,1.000000e+00,3.918000e+00


---

**Notes:** 
* target column ```fare_amount``` 
  * *min* value is negative. That means that some entities in the data has negative price for taxi ride what is an obvious error and can lead to wrong training of the model
  * *max* value is pretty high and we should put some attention on it
* ```passenger_count``` column *min* is zero and *max* is 9 passengers both of which are impossible
* ```distance``` column: 
  * *min* value is zero what is senseless
  * *max* value is really huge distances (12595km ~  distance from Anchorage, Alaska to Santiago de Chile, Chile by air! and obviously can't be covered by car)

**Conclusion:** closely observe
* ```fare_amount``` column to deal with negative values, check max fares and probable zero values
* ```passenger_count``` column to check that number of passengers per cab is realistic
* ```distance``` column to check for zero and huge values

---

### Target Feature Values


As it was mentioned earlier, we need to get rid of negative values in the target column ```fare_amount```. We also should check if there is any zero values in the target column as far as it brings no info for the future model and can be counted and treated as missing values.


---

In [21]:
df.fare_amount.describe()

count    1.027921e+06
mean     1.133004e+01
std      9.746336e+00
min     -4.490000e+01
25%      6.000000e+00
50%      8.500000e+00
75%      1.250000e+01
max      5.000000e+02
Name: fare_amount, dtype: float64

In [22]:
df[df.fare_amount<=0].shape #amount of negative and zero values for fare_amount column

(60, 10)

---

As we can see there are some negative and zero values in ```fare_amount``` column. These amount is not high in comparison with all entities in the data (~0.006%), we can skip these entities with dropping the whole row.

---

In [24]:
df = df[df.fare_amount>0]
print('df.shape: ', df.shape)
df.head()

df.shape:  (1027861, 10)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999


In [25]:
df[df.fare_amount<=0].shape

(0, 10)



---

No entities with negative or null ride price left.

Let's now check max values of fare amount. 

---



In [33]:
df[df.fare_amount>300]

,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
101885,500.00,1,2011,Fall,September,12,Monday,9,1,0.000
233873,450.00,1,2012,Fall,October,28,Sunday,14,0,0.692
247669,495.00,1,2015,Spring,May,12,Tuesday,13,0,140.515
287636,450.00,1,2015,Spring,March,11,Wednesday,16,1,0.002
329008,450.00,1,2011,Summer,July,29,Friday,14,0,0.001
361790,400.00,1,2011,Spring,May,5,Thursday,8,1,0.001
451970,400.00,1,2011,Fall,October,8,Saturday,5,0,0.009
951800,347.54,1,2015,Spring,March,24,Tuesday,17,1,84.725
1004265,325.00,1,2012,Winter,February,2,Thursday,23,0,0.276




---

After short analysis we can see there are just 2 entities with fare more than $300 that could be potentially realistic. That's why we are gonna just skip all of these entities.


---



In [34]:
df.drop(index=df[df.fare_amount>300].index.to_list(), inplace=True)
print('df.shape: ', df.shape)
df.head()

df.shape:  (1027852, 10)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999


In [35]:
df.fare_amount.describe()

count    1.027852e+06
mean     1.132730e+01
std      9.667486e+00
min      1.000000e-02
25%      6.000000e+00
50%      8.500000e+00
75%      1.250000e+01
max      2.870800e+02
Name: fare_amount, dtype: float64



---

Now we can see that *min* of ```fare_amount``` is $0.01 what is still pretty low and seems erroneous. 

---



In [40]:
df[df.fare_amount<1]

,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
2780,0.01,1,2015,Spring,May,1,Friday,15,1,0.222
90010,0.01,1,2015,Winter,February,19,Thursday,6,0,0.171
194168,0.01,2,2015,Winter,February,27,Friday,15,1,6.975
217966,0.01,1,2015,Winter,January,14,Wednesday,16,1,0.005
281140,0.01,1,2015,Spring,April,13,Monday,21,0,0.000
671865,0.01,1,2010,Winter,February,2,Tuesday,17,1,1.701
843937,0.01,1,2015,Spring,March,29,Sunday,10,1,0.299
1030274,0.01,1,2015,Spring,March,17,Tuesday,23,0,0.001
1046329,0.01,2,2015,Winter,January,5,Monday,21,0,0.004




---

There are also not so many entities with this fare_amount value, so we will drop them off as well.

---



In [41]:
df.drop(index=df[df.fare_amount<1].index.to_list(), inplace=True)
print('df.shape: ', df.shape)
df.head()

df.shape:  (1027843, 10)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999


In [42]:
df.fare_amount.describe()

count    1.027843e+06
mean     1.132739e+01
std      9.667471e+00
min      2.500000e+00
25%      6.000000e+00
50%      8.500000e+00
75%      1.250000e+01
max      2.870800e+02
Name: fare_amount, dtype: float64



---

Finally, ```fare_amount``` distribution looks realistic.

---





### Passenger_count Feature Values

Let's check how many unique values ```passenger_count``` column has.

---

In [46]:
df.passenger_count.unique()

array([1, 2, 3, 6, 5, 4, 0, 9])

---

With assumption that the largest car that can work in taxi is a mini van with 6 possible passengers, we can determine that ```passenger_count``` column should have values in range (1,6).

As we can see the data has 2 additional number of passengers: 0 and 9. Let's see how many entities have these values.

---

In [47]:
print('0 passengers: {}\n9 passengers: {}'.format(df[df.passenger_count==0].shape[0], df[df.passenger_count==9].shape[0]))

0 passengers: 3639
9 passengers: 1





---

As shown there are just 1 entity with more passengers than 6, we can drop this entity.

But there are a lot entities with 0 passengers. As long as taxi ride can't be without a passenger, we will place 1 passenger to each of those rides.

---



In [48]:
df = df[df.passenger_count<7] #drop rows with passenger_count > 6
df.passenger_count.replace(0,1, inplace=True) #replacing 0 with 1

print('df.shape: ', df.shape)
df.head()

df.shape:  (1027842, 10)


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999


In [49]:
df.passenger_count.describe()

count    1.027842e+06
mean     1.688796e+00
std      1.304760e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      6.000000e+00
Name: passenger_count, dtype: float64



---

```passenger_count``` column has *min* 1 passenger and *max* 6 passengers.

---



### Distance Feature Values

In [50]:
df.distance.describe()

count    1.027842e+06
mean     4.395168e+00
std      7.550584e+01
min      0.000000e+00
25%      1.254000e+00
50%      2.153000e+00
75%      3.918000e+00
max      1.259472e+04
Name: distance, dtype: float64



---

We got some zero values and some really huge distances. What makes me thinl there are some mistaken coordinates in the data which shouldn't be used in model training. That's why I will drop entities connected with them.


Let's start with huge distances. For this we need to define the edge after which the distance is way to big for taxi ride. Let's look at 99% percentile of ```distance``` column.

---



In [64]:
df.distance.quantile(.99944)

60.798907359997976



---

Now let's see what's going on with the fare around this distance. For this we will sort the data by ```distance``` column.


---



In [57]:
df.sort_values(by=['distance'], ascending=False)[['fare_amount','year','distance']][df.distance<80][df.distance>60]

<ipython-input-57-5bef68967df8>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.sort_values(by=['distance'], ascending=False)[['fare_amount','year','distance']][df.distance<80][df.distance>60]


,fare_amount,year,distance
1020302,36.50,2012,78.541
477723,7.00,2013,77.761
47520,49.80,2011,77.623
862121,7.00,2012,74.589
217224,245.41,2015,69.862
519943,52.00,2013,69.741
1006251,200.00,2015,69.015
881680,5.30,2011,67.663
405310,4.90,2011,66.736
224159,201.00,2013,65.543




---

After short analysis we can notide that entities with distance more than 60km have disproportional fare amount. That's why 60km is going to be our distance edge above which we will drop all entities.

---



In [65]:
df[df.distance>60].shape

(577, 10)

---

In comparison with all entities in the data there are not so many values that don't fit our condition (<0.06%) so we can skip these entities with dropping the whole row.

---

In [66]:
df = df[df.distance<60]
print('df.shape: ', df.shape)

df.shape:  (1027265, 10)




---

Now let's deal with zero values in ```distance``` column. 

---



In [37]:
df[df.distance==0].shape

(11495, 10)

In [68]:
df[df.distance==0][['fare_amount', 'distance']].head()

,fare_amount,distance
105,52.0,0.0
191,6.5,0.0
270,7.5,0.0
290,6.9,0.0
396,12.0,0.0




---

As shown we got rides with zero distance but not zero fare amount. That's why we can make an assumption that data was colllected not correctly. As long as we can't get info to fill these gaps from any source and this can influent future model we will drop the whole entity connected to particular zero (about 1% of the data).

---



In [69]:
df = df[df.distance > 0]
print('df.shape: ', df.shape)
df.head()

df.shape:  (1015772, 10)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999


In [70]:
df.distance.describe()

count    1.015772e+06
mean     3.350592e+00
std      3.589155e+00
min      1.000000e-03
25%      1.279000e+00
50%      2.177000e+00
75%      3.945250e+00
max      5.992100e+01
Name: distance, dtype: float64



---

*min* value is still pretty low - 1m. Taxi ride with distance less than 100m at least doesn't make sense and seems erroneous. And as tradition, I will drop these entities. 

---



In [80]:
df[df.distance<0.01]

,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
184,4.5,2,2013,Winter,February,6,Wednesday,14,0,0.004
616,16.0,1,2013,Fall,October,5,Saturday,20,0,0.003
1147,9.5,1,2015,Winter,January,11,Sunday,22,0,0.007
1494,70.0,1,2009,Fall,September,1,Tuesday,14,0,0.003
1750,2.5,1,2009,Fall,October,7,Wednesday,16,1,0.006
...,...,...,...,...,...,...,...,...,...,...
1047002,50.0,1,2010,Summer,July,14,Wednesday,14,0,0.003
1047247,50.0,1,2010,Summer,August,27,Friday,3,0,0.003
1047581,2.5,3,2009,Winter,February,21,Saturday,19,1,0.004
1047896,7.7,1,2010,Summer,July,28,Wednesday,9,1,0.001


In [81]:
df.drop(index=df[df.distance<0.01].index.to_list(), inplace=True)
print('df.shape: ', df.shape)
df.head()

df.shape:  (1013818, 10)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999




---

Finally I don't have too low and too huge distance entities in the data.

---



### Duplicates


Also to avoid false influence on the result by same data, let's check it on duplicates.

---

In [82]:
df.nunique() #Count number of distinct elements in specified axis

fare_amount         2028
passenger_count        6
year                   7
season                 4
month                 12
day                   31
day_name               7
hour                  24
rush_hour              2
distance           23124
dtype: int64

In [85]:
df[df.duplicated()] #count sum of  boolean Series denoting duplicate rows

,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
33777,4.9,1,2012,Winter,February,8,Wednesday,18,1,1.185
117115,6.1,1,2011,Winter,January,22,Saturday,10,1,1.534
176420,8.0,1,2013,Spring,March,23,Saturday,0,0,2.076
192035,7.7,1,2011,Summer,August,26,Friday,10,1,2.224
236606,7.3,1,2010,Summer,July,31,Saturday,14,0,2.065
...,...,...,...,...,...,...,...,...,...,...
1019115,6.1,1,2011,Spring,March,23,Wednesday,10,1,1.680
1037363,13.0,1,2014,Winter,January,31,Friday,23,0,4.540
1043864,6.0,1,2014,Winter,December,3,Wednesday,19,1,0.927
1045623,5.0,1,2015,Spring,April,25,Saturday,9,1,0.943




---

There is no duplicated rows in the data.

---



## Feature Extracting Function

After creating and training model we will have to apply it on the test data to get predictions. To do so we will need to convert test data  to the same format as we did with train data. For this purpose lets create a function that will do it for us:
* drop ```key``` column
* convert ```pickup_datetime``` into datetime data type
* create ```year```, ```season```, ```month```, ```day```, ```day_name```, ```hour```, ```rush_hour``` out of ```pickup_datetime```
* create ```distance``` column out of coordinate columns

---

In [41]:
def clean_data(df):

  df.drop(columns = ['key'], inplace=True) #drop key column

  df.pickup_datetime = pd.to_datetime(df.pickup_datetime) #convert dtype to datetime

  #creating year, season, month, day, day_name, hour columns
  df['year'] = df.pickup_datetime.dt.year #year
  seasons = ['Winter', 'Spring', 'Summer', 'Fall'] 
  df['season'] = [seasons[i-1] for i in (df.pickup_datetime.dt.month%12// 3 + 1).values] #season
  df['month'] = df.pickup_datetime.dt.month_name() #month
  df['day'] = df.pickup_datetime.dt.day #day
  df['day_name'] = df.pickup_datetime.dt.day_name() #day name
  df['hour'] = df.pickup_datetime.dt.hour #hour
  
  #creating rush hour column
  rush_hour = []
  for i in df.hour.values:
    if i in range(7,11):
      rush_hour.append(1)
    elif i in range(15,20):
      rush_hour.append(1)
    else:
      rush_hour.append(0)
  df['rush_hour'] = rush_hour # rush hour
  
  df.drop(columns=['pickup_datetime'], inplace=True) # drop donor pickup_datetime column

  #creating distance column
  distance = []
  for i in range(df.shape[0]):
    coordA = [df.pickup_latitude.iloc[i], df.pickup_longitude.iloc[i]]
    coordB = [df.dropoff_latitude.iloc[i], df.dropoff_longitude.iloc[i]]
    distance.append(round(float(great_circle(coordA, coordB).kilometers), 3))  
  df['distance'] = distance #create a column with distance for each ride
  
  df.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], inplace=True) #drop donor columns

  return df

## Save Clean Data

After finishing process of cleaning data let's save the result to avoid repeating the whole thing in the future.

---

In [426]:
df.to_csv('clean_train.csv', index=False)

# Data Analysis & Feature Selection

## Load cleaned data

In [42]:
df = pd.read_csv('https://raw.githubusercontent.com/pe44enka/TaxiFarePrediction/master/data/clean_train.csv')
print('df.shape: ', df.shape)
df.head()

df.shape:  (1015798, 10)


,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
0,4.5,1,2009,Summer,June,15,Monday,17,1,1.031
1,16.9,1,2010,Winter,January,5,Tuesday,16,1,8.450
2,5.7,2,2011,Summer,August,18,Thursday,0,0,1.390
3,7.7,1,2012,Spring,April,21,Saturday,4,0,2.799
4,5.3,1,2010,Spring,March,9,Tuesday,7,1,1.999


## Exploratory Data Analysis

In [43]:
df.describe(include='all')

,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
count,1.015798e+06,1.015798e+06,1.015798e+06,1015798,1015798,1.015798e+06,1015798,1.015798e+06,1.015798e+06,1.015798e+06
unique,NaN,NaN,NaN,4,12,NaN,7,NaN,NaN,NaN
top,NaN,NaN,NaN,Spring,May,NaN,Friday,NaN,NaN,NaN
freq,NaN,NaN,NaN,283826,95202,NaN,156715,NaN,NaN,NaN
mean,1.132407e+01,1.689175e+00,2.011745e+03,NaN,NaN,1.569651e+01,NaN,1.351023e+01,4.351948e-01,3.351284e+00
std,9.610758e+00,1.304941e+00,1.864025e+00,NaN,NaN,8.681503e+00,NaN,6.512098e+00,4.957827e-01,3.595385e+00
min,1.000000e-02,1.000000e+00,2.009000e+03,NaN,NaN,1.000000e+00,NaN,0.000000e+00,0.000000e+00,1.000000e-03
25%,6.000000e+00,1.000000e+00,2.010000e+03,NaN,NaN,8.000000e+00,NaN,9.000000e+00,0.000000e+00,1.279000e+00
50%,8.500000e+00,1.000000e+00,2.012000e+03,NaN,NaN,1.600000e+01,NaN,1.400000e+01,0.000000e+00,2.177000e+00
75%,1.250000e+01,2.000000e+00,2.013000e+03,NaN,NaN,2.300000e+01,NaN,1.900000e+01,1.000000e+00,3.946000e+00


In [44]:
df[df.fare_amount>400]

,fare_amount,passenger_count,year,season,month,day,day_name,hour,rush_hour,distance
226516,450.0,1,2012,Fall,October,28,Sunday,14,0,0.692
278647,450.0,1,2015,Spring,March,11,Wednesday,16,1,0.002
318784,450.0,1,2011,Summer,July,29,Friday,14,0,0.001
